In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image


In [2]:
BASE_PATH = r"E:\people-analytics\data\PETA\PETA dataset"
SUBSET = "CUHK"
IMAGE_DIR = os.path.join(BASE_PATH, SUBSET, "archive")

SELECTED_ATTRS = [
    "personalMale",
    "personalFemale",
    "personalLess30",
    "personalLess45",
    "upperBodyCasual",
    "upperBodyFormal",
    "upperBodyJacket",
    "upperBodyTshirt",
    "lowerBodyJeans",
    "lowerBodyTrousers",
    "carryingBackpack",
    "carryingMessengerBag"
]


In [3]:
import pandas as pd

LABEL_PATH = os.path.join(IMAGE_DIR, "Label.txt")

records = []

with open(LABEL_PATH, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 2:
            continue

        image_name = parts[0]
        attrs = parts[1:]
        image_path = os.path.join(IMAGE_DIR, image_name)

        if not os.path.exists(image_path):
            continue

        record = {"image_path": image_path}
        for attr in SELECTED_ATTRS:
            record[attr] = 1 if attr in attrs else 0

        records.append(record)

df = pd.DataFrame(records)


In [4]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [5]:
class PedestrianDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.label_cols = df.columns[1:]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["image_path"]).convert("RGB")
        image = self.transform(image)

        labels = torch.tensor(
            row[self.label_cols].values, dtype=torch.float32
        )

        return image, labels


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_loader = DataLoader(
    PedestrianDataset(train_df, transform),
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    PedestrianDataset(val_df, transform),
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)


In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda
